###  Profiling and Timing

In [ ]:
import timeit
import time
import cProfile
import random
from memory_profiler import profile

# 1. timeit basic usage
time_taken = timeit.timeit('sum(range(10000))', number=1000)
print(f"Time taken: {time_taken:.4f} seconds")

# 2. List vs Generator comparison
list_time = timeit.timeit('[x*x for x in range(1000000)]', number=10)
gen_time = timeit.timeit('(x*x for x in range(1000000))', number=10)
print(f"List comprehension: {list_time:.4f}s, Generator: {gen_time:.4f}s")

# 3. cProfile usage (will show output when run)
def profile_me():
    total = 0
    for i in range(10000):
        total += i*i
    return total

cProfile.run('profile_me()')

# 4. line_profiler example (requires separate installation)
@profile
def memory_intensive():
    data = [random.random() for _ in range(100000)]
    result = [x * 2 for x in data]
    return sum(result)

# 5. Manual timing
start = time.time()
sum(range(1000000))
end = time.time()
print(f"Manual timing: {end - start:.4f} seconds")

# 6. Benchmark sorting
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]

data = [random.randint(0, 1000) for _ in range(1000)]
sorted_time = timeit.timeit('sorted(data)', globals=globals(), number=100)
bubble_time = timeit.timeit('bubble_sort(data.copy())', globals=globals(), number=100)
print(f"Built-in sorted: {sorted_time:.4f}s, Bubble sort: {bubble_time:.4f}s")

# 7. memory_profiler usage
@profile
def create_large_list():
    return [i**2 for i in range(100000)]

large_list = create_large_list()

### Lazy Evaluation and Efficiency


In [ ]:
import sys
import itertools

# 1. Large file reading with generator
def read_large_file(filename):
    with open(filename) as f:
        for line in f:
            yield line.strip()

# 2. Memory comparison
numbers = range(1000000)
list_size = sys.getsizeof([x for x in numbers])
gen_size = sys.getsizeof(x for x in numbers)
print(f"List size: {list_size:,} bytes, Generator size: {gen_size:,} bytes")

# 3. Lazy CSV filter
def filter_csv(filename, condition):
    with open(filename) as f:
        reader = csv.reader(f)
        header = next(reader)
        yield header
        for row in reader:
            if condition(row):
                yield row

# 4. Short-circuiting with any()
big_list = range(100000000)
divisible_by_99 = any(x % 99 == 0 for x in big_list)
print(f"Contains divisible by 99: {divisible_by_99}")

# 5. itertools.islice example
first_10_lines = itertools.islice(read_large_file('bigfile.txt'), 10)
print(list(first_10_lines))

# 6. Avoid temporary lists
total = sum(x for x in range(1000000))
print(f"Sum: {total}")

# 7. Streaming file copy
def stream_copy(source, dest):
    with open(source) as src, open(dest, 'w') as dst:
        for line in src:
            dst.write(line)

# 8. Yield vs return
def get_numbers_list(n):
    result = []
    for i in range(n):
        result.append(i * 2)
    return result

def get_numbers_gen(n):
    for i in range(n):
        yield i * 2

# Example usage
if __name__ == "__main__":
    # Create sample files
    with open('bigfile.txt', 'w') as f:
        f.writelines(f"Line {i}\n" for i in range(1000))

    with open('data.csv', 'w') as f:
        f.write("id,name,value\n")
        f.writelines(f"{i},Item{i},{i*10}\n" for i in range(100))

    # Test the generators
    print("First 5 lines:")
    for i, line in enumerate(read_large_file('bigfile.txt')):
        if i >= 5:
            break
        print(line)

    print("\nFiltered CSV (value > 50):")
    filtered = filter_csv('data.csv', lambda row: int(row[2]) > 50)
    for row in itertools.islice(filtered, 5):
        print(row)

    # Memory comparison
    list_nums = get_numbers_list(1000000)
    gen_nums = get_numbers_gen(1000000)
    print(f"\nList memory: {sys.getsizeof(list_nums):,} bytes")
    print(f"Generator memory: {sys.getsizeof(gen_nums):,} bytes")

### Debugging Tools and Practices

In [ ]:
import pdb
import traceback
import logging
import warnings
from functools import wraps

# Configure structured logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# 1. pdb.set_trace()
def debug_function(x):
    y = x * 2
    pdb.set_trace()  # Execution pauses here
    return y + 3

# 2. breakpoint() (Python 3.7+)
def calculate_stats(data):
    total = sum(data)
    breakpoint()  # Modern equivalent of pdb.set_trace()
    return total / len(data)

# 3. traceback module
def risky_operation():
    try:
        1 / 0
    except Exception:
        print("Error details:\n", traceback.format_exc())

# 4. Structured logging decorator
def log_calls(func):
    @wraps(func
    def wrapper(*args, **kwargs):
        logger.debug(f"Entering {func.__name__}")
        try:
            result = func(*args, **kwargs)
            logger.debug(f"Exiting {func.__name__}")
            return result
        except Exception as e:
            logger.error(f"Exception in {func.__name__}: {e}")
            raise
    return wrapper

# 5. warnings module
def deprecated_feature():
    warnings.warn("This feature will be removed", DeprecationWarning)
    return "old result"

# 6. Verbose exceptions
def parse_number(s):
    try:
        return float(s)
    except Exception as e:
        print(f"Error type: {type(e).__name__}, Message: {e}")
        return None

# 7. Debug recursive calls
def factorial(n, level=0):
    logger.debug(f"{'  '*level}Calling factorial({n})")
    if n <= 1:
        return 1
    return n * factorial(n-1, level+1)

# 8. Fail loud after logging
def critical_operation():
    try:
        # Simulate error
        raise ValueError("Critical data missing")
    except Exception as e:
        logger.exception("Operation failed")
        raise  # Re-raise after logging

# Example usage
if __name__ == "__main__":
    debug_function(5)  # Will pause in pdb
    calculate_stats([1,2,3])  # Will pause at breakpoint
    risky_operation()
    deprecated_feature()
    parse_number("abc")
    factorial(5)

    try:
        critical_operation()
    except ValueError:
        print("Handled the critical error")

### Design for Observability

In [ ]:
import logging
import time
import os
import psutil
from functools import wraps
from random import random

# Configuration
DEBUG = os.getenv('DEBUG', 'False').lower() == 'true'
logging.basicConfig(level=logging.DEBUG if DEBUG else logging.INFO)
logger = logging.getLogger(__name__)

# Metrics dictionary
metrics = {
    'calls': {},
    'timers': {},
    'counters': {},
    'errors': {}
}

def log_with_context(user_id=None):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            extra = {'user_id': user_id, 'func': func.__name__}

            try:
                result = func(*args, **kwargs)
                duration = time.time() - start_time

                metrics['calls'][func.__name__] = metrics['calls'].get(func.__name__, 0) + 1
                metrics['timers'][func.__name__] = metrics['timers'].get(func.__name__, 0) + duration

                logger.info(f"Completed {func.__name__} in {duration:.4f}s", extra=extra)
                return result

            except Exception as e:
                error_id = f"ERR-{int(random()*10000):04d}"
                metrics['errors'][error_id] = metrics['errors'].get(error_id, 0) + 1
                logger.error(f"Error {error_id} in {func.__name__}: {str(e)}", extra=extra)
                raise

        return wrapper
    return decorator

@log_with_context(user_id="user123")
def process_data(data):
    """Example function with logging and timing"""
    time.sleep(0.1)  # Simulate work
    if random() > 0.8:
        raise ValueError("Random processing error")
    return data * 2

def health_check():
    """System health check"""
    status = {
        'memory': psutil.virtual_memory().percent,
        'cpu': psutil.cpu_percent(),
        'load': os.getloadavg(),
        'disk': psutil.disk_usage('/').percent
    }
    logger.info("System status", extra={'status': status})
    return status

def get_metrics():
    """Return collected metrics"""
    return metrics

def print_resource_usage():
    """Print current resource usage"""
    mem = psutil.virtual_memory()
    print(f"Memory: {mem.used/1024/1024:.1f}MB used ({mem.percent}%)")
    print(f"CPU: {psutil.cpu_percent()}%")
    print(f"Load: {os.getloadavg()}")

if __name__ == "__main__":
    # Example usage
    for i in range(5):
        try:
            result = process_data(i)
            print(f"Result: {result}")
        except ValueError as e:
            print(f"Failed: {e}")

    # System checks
    health_status = health_check()
    print("\nHealth check:", health_status)

    # Metrics
    print("\nMetrics:")
    for k, v in get_metrics().items():
        print(f"{k}: {v}")

    # Current resources
    print("\nCurrent resources:")
    print_resource_usage()